### Setup

First, import the required packages.

Next, specify some important variables:
* `platform_address`: The address of the Yugabyte Platform API
* `platform_api_key`: The API key used to authenticate with the Platform API

Finally, open a HTTP connection to the Yugabyte Platform API.

In [13]:
import os
import http.client
import json
from pprint import pprint

platform_address = os.getenv('API_BASE_URL', "portal.dev.yugabyte.com")
platform_api_key = os.getenv('YB_API_KEY')

conn = http.client.HTTPConnection(f"{platform_address}")

headers = {
  'Content-Type': "application/json",
  'X-AUTH-YW-API-TOKEN': f"{platform_api_key}"
}

### Get Session Info

Make an API call to session endpoint to determine customer UUID.

In [14]:
conn.request("GET", "/api/v1/session_info", headers=headers)

res = conn.getresponse()
data = res.read()
session_info = json.loads(data)
customer_uuid = session_info['customerUUID']

print('Customer UUID:\n%s' % customer_uuid)

Customer UUID:
11d78d93-1381-4d1d-8393-ba76f47ba7a6


### Get Provider UUID

Make an API call to the provider endpoint to determine provider UUID and regions.

In [16]:
url = f"/api/v1/customers/{customer_uuid}/providers"
conn.request("GET", url, headers=headers)
res = conn.getresponse()

provider_list = json.load(res)
for provider in provider_list:
    if provider['code'] == 'gcp' and provider['active'] == True:
        provider_uuid = provider['uuid']
        region_list = [region['uuid'] for region in provider['regions']]
        break

print('Provider UUID:\n%s' % provider_uuid)
print('Regions:\n[%s]' % ', '.join(region_list))

Provider UUID:
109e95b5-bf08-4a8f-a7fb-2d2866865e15
Regions:
[65d5719a-a87b-43c7-be53-c87e467ecf46]


### Get Access Key

Make an API call to the access key endpoint to determine access key for provider.

In [7]:
url = f"/api/v1/customers/{customer_uuid}/providers/{provider_uuid}/access_keys"
conn.request("GET", url, headers=headers)
res = conn.getresponse()

access_key_list = json.load(res)
access_key_code = access_key_list[0]['idKey']['keyCode']
print('Access Key:\n%s' % access_key_code)

Access Key:
yb-gcp-config-key


### Define Universe

In this example, we will create a GCP universe. Define a `Universe` object with the desired configuration.

In [8]:
new_universe = {
    'clusters': [
        {
            'clusterType': 'PRIMARY',
            'userIntent': {
                'universeName': 'my-gcp-universe',
                'providerType': 'gcp',
                'provider': provider_uuid,
                'regionList': region_list,
                'numNodes': 3,
                'replicationFactor': 3,
                'instanceType': 'n1-standard-1',
                'deviceInfo': {
                    'numVolumes': 1,
                    'storageType': 'Persistent',
                    'volumeSize': 375,
                },
                'assignPublicIP': True,
                'useTimeSync': True,
                'enableYSQL': True,
                'enableYEDIS': False,
                'enableNodeToNodeEncrypt': True,
                'enableClientToNodeEncrypt': True,
                'enableVolumeEncryption': False,
                'ybSoftwareVersion': '2.7.3.0-b80',
                'accessKeyCode': access_key_code,
                'tserverGFlags': {},
                'masterGFlags': {},
            }
        },
    ],
}

### Create Universe

Make API call to create new universe.

In [9]:
url = f"/api/v1/customers/{customer_uuid}/universes/clusters"
conn.request("POST", url, json.dumps(new_universe), headers)
res = conn.getresponse()
pprint(json.load(res))

{
  "taskUUID": "a3f597fa-dd48-42f6-a106-dd14c15343a2",
  "resourceUUID": "303cf5ac-a6f5-492b-b68c-7d4d9633c4cc"
}
